# A Dual Approach to Holistic Regression
## March 4th, 2021

In [5]:
using Random, Distributions
using LinearAlgebra
using Gurobi, JuMP
using DataFrames
using CSV
using StatsBase
using Plots
using ProgressBars
using Optim

In [7]:
# Create a gurobi model without the annoying academic license message
gurobi_env = Gurobi.Env()
function create_gurobi_model(; TimeLimit=-1, LogFile=nothing)
    model = Model(optimizer_with_attributes(() -> Gurobi.Optimizer(gurobi_env)));
    if TimeLimit >= 0
        println("Set Gurobi TimeLimit.")
        set_optimizer_attribute(model, "TimeLimit", TimeLimit)
    end
    if LogFile != nothing
        println("LogFile: $(LogFile).")
        set_optimizer_attribute(model, "LogFile", LogFile)
    else
        set_optimizer_attribute(model, "OutputFlag", 0)
    end
    set_optimizer_attribute(model, "NumericFocus", 3)
    #set_optimizer_attribute(model, "Threads", 4)
    return model
end;


--------------------------------------------
--------------------------------------------

Academic license - for non-commercial use only


____
## 0. Utils

In [9]:
function get_support(s)
    supp = similar(s, Int)
    count_supp = 1
    
    supp_c = similar(s, Int)
    count_supp_c = 1
    
    @inbounds for i in eachindex(s)
        supp[count_supp] = i
        supp_c[count_supp_c] = i
        is_zero = s[i] < 0.5
        count_supp += !is_zero
        count_supp_c += is_zero
    end
    return resize!(supp, count_supp-1), resize!(supp_c, count_supp_c-1)
end

get_support([0, 0, 1, 1, 0])

([3, 4], [1, 2, 5])

____
## 1. Generate Synthetic Data  

In [11]:
function generate_synthetic_data(n, p, k, NR; seed=-1)
    """
        n = num. of samples
        p = num. of features
        k = num. of non zero coefficients
        NR = noise ratio ~ σ_noise = NR * σ_y_true
    """
    if seed > 0
        Random.seed!(seed)
    end
    
    # Generate PD matrix
    A = randn(p, p)
    A = A'*A
    Σ = (A' + A)/2
    
    # Generate data X
    d = MvNormal(Σ)
    X = rand(d, n)'I
    
    # Split data
    index_train = 1:floor(Int, 0.5*n)
    index_val = floor(Int, 0.5*n)+1:floor(Int, 0.75*n)
    index_test = floor(Int, 0.75*n)+1:n
    
    X_train = X[index_train,:]
    X_val = X[index_val,:]
    X_test = X[index_test,:]
    
    # Center
    μ_train = [mean(X_train[:, j]) for j=1:p]
    for j=1:p
         X_train[:,j] = X_train[:,j] .- μ_train[j]
         X_val[:,j] = X_val[:,j] .- μ_train[j]
         X_test[:,j] = X_test[:,j] .- μ_train[j]
    end
    
    # Scale
    σ_train = [norm(X_train[:, j]) for j=1:p]
    for j=1:p
         X_train[:,j] = X_train[:,j]/σ_train[j]
         X_val[:,j] = X_val[:,j] ./ σ_train[j]
         X_test[:,j] = X_test[:,j] ./ σ_train[j]
    end
    
    # Generate β
    β = zeros(p)
    for j=1:k
        β[floor(Int, j*p/k)] = 1.0*rand([-1, 1])
    end
    
    # Noise
    ϵ = rand(Normal(0, std(X*β)*NR), n)
    
    # Target
    y_train = X_train*β + ϵ[index_train]
    y_val = X_val*β + ϵ[index_val]
    y_test = X_test*β + ϵ[index_test]
            
    return  (X_train, y_train), (X_val, y_val), (X_test, y_test), β
end

function get_t_α(n, p, α)
    return quantile(TDist(n-p), 1 - α/2)
end

function get_σ_X(X, y, γ)
    n, p = size(X)
    
    # Estimator σ
    M_inv = inv(I/γ + X'X)
    σ_tilde = sqrt((y'*(I - X*M_inv*X')*y)/(n-p))
    σ_X = σ_tilde * sqrt.(diag(M_inv))
    
    return σ_X
end

function get_R2(y_pred, y_true, y_train)
    SS_res = norm(y_true .- y_pred)
    SS_tot = norm(y_true .- mean(y_train))
    return 1 - (SS_res/SS_tot)^2
end

;

### /!\ t_α is already in σ_X /!\

In [13]:
# Parameters
n_train = 10000
n = 2*n_train
p = 500
k = 10
NR = 0.001
α = 0.05
t_α = get_t_α(n, p, α)
γ = 1.0

# Generate data
(X_p, y), _, _, β_true = generate_synthetic_data(n, p, k, NR, seed=2021);
σ_X_p = t_α * get_σ_X(X_p, y, γ); #t_α is already in σ_X

# Compute data in p dimensions
M_p = X_p'X_p
b_p = X_p'y

# Compute data in 2p dimensions
M = [M_p -M_p; -M_p  M_p]
b = [b_p; -b_p];
σ_X = [σ_X_p; σ_X_p] ;

## 2. Compute inner problems and gradients

### a. Compute g_s

In [15]:
function g_s(D_s, b_s, σ_X_s; GD=true)
    
    # Get length of support of s
    l = length(b_s)
    if l==0
        return zeros(0), 0.0
    end
    
    # Initial solution
    λ_s0 = zeros(l) .+ 1.0

    # Objective and gradient
    function fg!(F, G, λ_s)
        
        μ_s = λ_s .+ b_s
        β_s = D_s*μ_s
        
        if G != nothing
            G .= β_s .- σ_X_s
        end
        
        if F != nothing
            return -λ_s'σ_X_s + 0.5*μ_s'β_s
        end
    end
    
    lower = zeros(l)
    upper = [Inf for _ in 1:l]

    res = Optim.optimize(Optim.only_fg!(fg!), lower, upper, λ_s0, 
        Fminbox(GD ? GradientDescent() : LBFGS()))

    return Optim.minimizer(res), - Optim.minimum(res)
    
end;

In [17]:
function g_s_gurobi(D_s, b_s, σ_X_s, model)
    
    # Get length of support of s
    l = length(b_s)
    if l==0
        return zeros(0), 0.0
    end

    λ_s = model[:λ][1:l]
    μ_s = λ_s .+ b_s
    β_s = D_s*μ_s
    
    @objective(model, Max, λ_s'σ_X_s - 0.5*μ_s'β_s)
    
    optimize!(model)
    
    value.(λ_s), objective_value(model)
end;

In [19]:
function g_gurobi(supp, Z, D, b, σ_X, model)

    # Create DZ once
    DZ = D*Z
    
    λ = model[:λ]
    μ = b + λ

    @objective(model, Max, λ'*Z*σ_X - 0.5μ'*DZ*μ)
    
    optimize!(model)
    
    value.(λ)[supp], objective_value(model)
end;

### b. Compute ∇g_s

In [21]:
function ∇g_s(supp, supp_c, b, M, λ_s, D_s, σ_X_s)
    
    β_s = D_s*(b[supp] .+ λ_s)

    grad = zeros(2p)    
    
    grad[supp] = λ_s .* σ_X_s - (β_s .^ 2)/(2γ)
    grad[supp_c] = - 0.5*γ*(b[supp_c] - M[supp_c, supp]*β_s).^2
    
    return grad
    
end

∇g_s (generic function with 1 method)

In [23]:
function ∇g(supp, D, b, λ_s, σ_X)
    
    λ = zeros(2p)
    λ[supp] = λ_s
    
    grad = λ .* σ_X - ((D'*(b + λ)).^ 2)/(2γ)

    return grad
    
end

∇g (generic function with 1 method)

_____
## 3. Compare speed 

In [25]:
# Create s
s_true = vcat(β_true .> 0, β_true .< 0) .* 1 
supp, supp_c = get_support(s_true)

# Get projected variables
b_s = b[supp];
σ_X_s = σ_X[supp];
D_s = inv(I/γ + M[supp, supp]);

# Create model for g_s_gurobi
model_inner_g_s = create_gurobi_model();
@variable(model_inner_g_s, λ[1:k] >= 0)

# Create model for g_gurobi
model_inner_g = create_gurobi_model();
@variable(model_inner_g, λ[1:2p] >= 0);

Z = Diagonal(s_true);
D = inv(I/γ + Z*M);

### c. Compare g_s

In [27]:
λ_s_GD, g_s_GD = g_s(D_s, b_s, σ_X_s; GD=true)
λ_s_LBFGS, g_s_LBFGS = g_s(D_s, b_s, σ_X_s; GD=false)
λ_s_guro_s, g_s_guro_s = g_s_gurobi(D_s, b_s, σ_X_s, model_inner_g_s)
λ_s_guro, g_s_guro = g_gurobi(supp, Z, D, b, σ_X, model_inner_g)

# Compare objective values
println("GD: ",g_s_GD)
println("LBFGS: ",g_s_LBFGS)
println("Gurobi (g_s): ", g_s_guro_s)
println("Gurobi (g): ", g_s_guro)

# Compare λ
hcat(λ_s_GD, λ_s_LBFGS, λ_s_guro_s, λ_s_guro)

GD: -2.518213415937815
LBFGS: -2.518213415937815
Gurobi (g_s): -2.518213423909655
Gurobi (g): -2.518213423909655


10×4 Array{Float64,2}:
 1.12422e-20  5.18997e-24  1.56856e-9  1.56856e-9
 5.9688e-20   8.80851e-21  1.99349e-9  1.99349e-9
 4.70724e-20  7.56529e-25  2.31321e-9  2.31321e-9
 4.84469e-20  2.87416e-25  2.0871e-9   2.0871e-9 
 5.54016e-20  2.51693e-20  1.54509e-9  1.54509e-9
 7.40073e-20  1.10237e-18  2.49854e-9  2.49854e-9
 4.89772e-20  4.86968e-19  1.45543e-9  1.45543e-9
 1.03933e-21  1.00852e-18  2.81061e-9  2.81061e-9
 3.51037e-22  1.24333e-18  3.07943e-9  3.07943e-9
 6.96499e-20  1.36221e-23  2.5306e-9   2.5306e-9 

In [29]:
# Computational time for D_s compared to D (given s)

total_time_D_s = 0
total_time_D = 0
for _ in 1:100
    
    total_time_D_s += @elapsed begin 
        supp, supp_c = get_support(s_true)
        b_s = b[supp];
        σ_X_s = σ_X[supp];
        D_s = inv(I/γ + M[supp, supp]);
    end
    
    total_time_D += @elapsed begin
        Z = Diagonal(s_true);
        D = inv(I/γ + Z*M);
    end
end

println("Compute D_s: ", total_time_D_s)
println("Compute D: ", total_time_D)

Compute D_s: 0.0030135059999999987
Compute D: 7.682329111000001


In [30]:
# Computational time of g_s and g for given D_s or D

total_time_GD = 0
total_time_LBFGS = 0
total_time_Gurobi_g_s = 0
total_time_Gurobi_g = 0

for _ in 1:100
    total_time_GD += @elapsed g_s(D_s, b_s, σ_X_s; GD=true)
    total_time_LBFGS += @elapsed g_s(D_s, b_s, σ_X_s; GD=false)
    total_time_Gurobi_g_s += @elapsed g_s_gurobi(D_s, b_s, σ_X_s, model_inner_g_s)
    total_time_Gurobi_g += @elapsed g_gurobi(supp, Z, D, b, σ_X, model_inner_g)
end

println("Optim.jl + GD: ",total_time_GD)
println("Optim.jl + LBFGS: ", total_time_LBFGS)
println("Gurobi g_s (model created outside): ", total_time_Gurobi_g_s)
println("Gurobi g (model created outside): ", total_time_Gurobi_g)

Optim.jl + GD: 0.04399083300000001
Optim.jl + LBFGS: 0.10428594
Gurobi g_s (model created outside): 0.10771602600000002
Gurobi g (model created outside): 76.20070682799998


### d. Compare ∇g_s

In [31]:
# Compare the gradient for different optimal lambdas

∇g_s_guro = ∇g_s(supp, supp_c, b, M, λ_s_guro, D_s, σ_X_s)
∇g_s_GD = ∇g_s(supp, supp_c, b, M, λ_s_GD, D_s, σ_X_s)
∇g_guro = ∇g(supp, D, b, λ_s_guro, σ_X)

println("|| ∇g_s_guro - ∇g_s_GD || =  ", norm(∇g_s_guro - ∇g_s_GD))
hcat(∇g_s_GD, ∇g_s_guro, ∇g_guro)[1:5, :]

|| ∇g_s_guro - ∇g_s_GD || =  2.0209223269900075e-9


5×3 Array{Float64,2}:
 -0.0286636   -0.0286636   -0.0286636 
 -0.00744848  -0.00744848  -0.00744848
 -0.0163148   -0.0163148   -0.0163148 
 -0.00435322  -0.00435322  -0.00435322
 -0.00978691  -0.00978691  -0.00978691

In [35]:
# Computational time for D_s compared to D (given s)

total_time_∇g_s = 0
total_time_∇g = 0

for _ in 1:100
    
    total_time_∇g_s += @elapsed begin 
        ∇g_s(supp, supp_c, b, M, λ_s_GD, D_s, σ_X_s)
    end
    
    total_time_∇g += @elapsed begin
        ∇g(supp, D, b, λ_s_GD, σ_X)
    end
end

println("Compute ∇g_s: ", total_time_∇g_s)
println("Compute ∇g: ", total_time_∇g)

Compute ∇g_s: 0.033499707000000004
Compute ∇g: 0.11862310499999999


_____
## 4. Compute Cutting plane algorithm